In [10]:
!pip install torchvision

You should consider upgrading via the 'D:\Pranav\active-learn\Scripts\python.exe -m pip install --upgrade pip' command.


In [120]:
import numpy as np
import torch
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import pickle, time
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, TensorDataset

device = (
    "cuda"
    if torch.cuda.is_available()
    else "cpu"
)


In [108]:
def unpickle(file):
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

class Classifier(nn.Module):
    def __init__(self):
        super(Classifier, self).__init__()
	# 3 input image channel, 6 output channels, 
	# 5x5 square convolution kernel
        self.conv1 = nn.Conv2d(3, 6, 5)
	# Max pooling over a (2, 2) window
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5) 
        self.fc1 = nn.Linear(16 * 5 * 5, 120)# 5x5 from image dimension
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [76]:
transform = transforms.Compose( # composing several transforms together
    [transforms.ToTensor(), # to tensor object
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]) # mean = 0.5, std = 0.5

batch_size = 4
num_workers = 0
trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                          shuffle=True, num_workers=0)

# load test data
testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4,
                                         shuffle=False, num_workers=0)

# put 10 classes into a set
classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship','truck')

0.6%


KeyboardInterrupt: 

In [116]:
classes = unpickle('./cifar/batches.meta')[b'label_names']
labels = []
data = []
test_labels = []
test_data = []
for i in range(1, 6):
    path = "./cifar/data_batch_" + str(i)
    labels += (unpickle(path)[b'labels'])
    for j in unpickle(path)[b'data']:
        data.append(j)
test_labels += unpickle("./cifar/test_batch")[b'labels']
for i in unpickle("./cifar/test_batch")[b'data']:
    test_data.append(i)
arr_data = np.stack(data)
arr_labels = np.stack(labels)
tensor_X = ((torch.from_numpy(arr_data)).to(torch.float32)).reshape([3,32,32,50000])
tensor_y = (torch.from_numpy(arr_labels)).to(torch.long)

tensor_test_X = ((torch.from_numpy(np.stack(test_data))).to(torch.float32)).reshape([3,32,32,10000])
tensor_test_y = (torch.from_numpy(np.stack(test_labels))).to(torch.long)

print(tensor_X.shape)


torch.Size([3, 32, 32, 50000])


In [110]:
train_dataset = TensorDataset(tensor_X, tensor_y)
train_loader = DataLoader(train_dataset, batch_size=32)

test_dataset = TensorDataset(tensor_test_X, tensor_test_y)
test_loader = DataLoader(test_dataset, batch_size=32 )
#having initialised dataset and dataloader

#initialise model, loss fn., optimiser
model = Classifier()
criterion = nn.NLLLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)
num_epochs = 5

In [117]:
start = time.time()
best_acc = 0.0
for epoch in range(num_epochs):
    epoch_start = time.time()
    print(f"Epoch {epoch}")
    model.train()
    train_loss = 0.0
    train_acc = 0.0


    for (inputs, labels) in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()



Epoch 0
Epoch 1
Epoch 2
Epoch 3
Epoch 4


In [119]:
model.eval()

# Initialize variables to track performance
total_samples = 0
correct_predictions = 0

# Disable gradient computations
with torch.no_grad():
    for inputs, targets in test_loader:
        outputs = model(inputs)

        # Get the predicted class (the one with the highest probability)
        _, predicted = torch.max(outputs.data, 1)
        print(predicted)
        # Update total and correct counts
        total_samples += targets.size(0)
        correct_predictions += (predicted == targets).sum().item()

# Calculate accuracy
accuracy = 100 * correct_predictions / total_samples
print(f'Accuracy of the model on the test data: {accuracy:.2f}%')


tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0])
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0])
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0])
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0])
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0])
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0])
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0])
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0])
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0,